# **Chapter 22: Local Data Persistence**

---

## **Learning Objectives**

By the end of this chapter, you will be able to:

- Implement persistent storage using `SharedPreferences` for simple key-value data
- Design and manage SQLite databases using `sqflite` for relational data structures
- Utilize type-safe ORMs (Floor/Drift) to eliminate boilerplate and SQL injection risks
- Implement high-performance NoSQL storage with Hive for unstructured data
- Leverage ObjectBox and Isar for complex object graphs and high-throughput scenarios
- Perform secure file system operations using `path_provider`
- Architect offline-first synchronization strategies between local and remote data

---

## **Prerequisites**

- Completed Chapter 6: Asynchronous Programming (Futures, Streams)
- Completed Chapter 20: Authentication & Security (understanding of secure vs. insecure storage)
- Understanding of SQL basics (tables, indices, relationships)
- Basic knowledge of data serialization (JSON)

---

## **22.1 SharedPreferences**

`SharedPreferences` provides simple persistent storage for primitive data types (bool, int, double, String, List<String>). It is **not** secure and should never store passwords, tokens, or PII (Personally Identifiable Information).

### **Basic Implementation**

```dart
// preferences_service.dart
// Wrapper around SharedPreferences with type safety and migration support

import 'package:shared_preferences/shared_preferences.dart';
import 'dart:convert';

/// Service for managing non-sensitive user preferences
/// Storage is asynchronous and persists across app restarts
class PreferencesService {
  // Singleton pattern for single instance across app
  static final PreferencesService _instance = PreferencesService._internal();
  factory PreferencesService() => _instance;
  PreferencesService._internal();
  
  SharedPreferences? _prefs;
  
  /// Initialize the service
  /// Must be called before using any other methods
  /// Typically called in main() before runApp()
  Future<void> init() async {
    _prefs = await SharedPreferences.getInstance();
  }
  
  /// Generic getter with null safety
  /// [T] must be one of: bool, int, double, String, List<String>
  T? get<T>(String key) {
    _checkInitialized();
    
    // Type-specific retrieval
    if (T == bool) return _prefs!.getBool(key) as T?;
    if (T == int) return _prefs!.getInt(key) as T?;
    if (T == double) return _prefs!.getDouble(key) as T?;
    if (T == String) return _prefs!.getString(key) as T?;
    if (T == List<String>) return _prefs!.getStringList(key) as T?;
    
    throw ArgumentError('Type $T not supported by SharedPreferences');
  }
  
  /// Generic setter with type safety
  Future<bool> set<T>(String key, T value) async {
    _checkInitialized();
    
    if (value is bool) return await _prefs!.setBool(key, value);
    if (value is int) return await _prefs!.setInt(key, value);
    if (value is double) return await _prefs!.setDouble(key, value);
    if (value is String) return await _prefs!.setString(key, value);
    if (value is List<String>) return await _prefs!.setStringList(key, value);
    
    throw ArgumentError('Type ${value.runtimeType} not supported');
  }
  
  /// Store complex objects as JSON strings
  /// [T] must implement toJson() method
  Future<bool> setObject<T>(String key, T object, Map<String, dynamic> Function(T) toJson) async {
    final jsonString = jsonEncode(toJson(object));
    return await set<String>(key, jsonString);
  }
  
  /// Retrieve complex objects from JSON
  T? getObject<T>(String key, T Function(Map<String, dynamic>) fromJson) {
    final jsonString = get<String>(key);
    if (jsonString == null) return null;
    
    try {
      final map = jsonDecode(jsonString) as Map<String, dynamic>;
      return fromJson(map);
    } catch (e) {
      print('Error parsing object from preferences: $e');
      return null;
    }
  }
  
  /// Remove specific key
  Future<bool> remove(String key) async {
    _checkInitialized();
    return await _prefs!.remove(key);
  }
  
  /// Clear all preferences (use with caution)
  Future<bool> clear() async {
    _checkInitialized();
    return await _prefs!.clear();
  }
  
  /// Check if key exists
  bool containsKey(String key) {
    _checkInitialized();
    return _prefs!.containsKey(key);
  }
  
  /// Get all keys
  Set<String> getKeys() {
    _checkInitialized();
    return _prefs!.getKeys();
  }
  
  /// Atomic update: Read, modify, write with conflict protection
  /// [updater] function receives current value and returns new value
  Future<bool> update<T>(String key, T Function(T? current) updater) async {
    final current = get<T>(key);
    final updated = updater(current);
    return await set<T>(key, updated);
  }
  
  void _checkInitialized() {
    if (_prefs == null) {
      throw StateError('PreferencesService not initialized. Call init() first.');
    }
  }
}

/// Example model for complex object storage
class UserSettings {
  final String theme;
  final int fontSize;
  final bool notificationsEnabled;
  final List<String> favoriteTags;
  
  UserSettings({
    required this.theme,
    required this.fontSize,
    required this.notificationsEnabled,
    required this.favoriteTags,
  });
  
  Map<String, dynamic> toJson() => {
    'theme': theme,
    'fontSize': fontSize,
    'notificationsEnabled': notificationsEnabled,
    'favoriteTags': favoriteTags,
  };
  
  factory UserSettings.fromJson(Map<String, dynamic> json) => UserSettings(
    theme: json['theme'],
    fontSize: json['fontSize'],
    notificationsEnabled: json['notificationsEnabled'],
    favoriteTags: List<String>.from(json['favoriteTags']),
  );
  
  // Copy with method for immutable updates
  UserSettings copyWith({
    String? theme,
    int? fontSize,
    bool? notificationsEnabled,
    List<String>? favoriteTags,
  }) => UserSettings(
    theme: theme ?? this.theme,
    fontSize: fontSize ?? this.fontSize,
    notificationsEnabled: notificationsEnabled ?? this.notificationsEnabled,
    favoriteTags: favoriteTags ?? this.favoriteTags,
  );
}
```

**Explanation:**

- **Singleton Pattern**: `PreferencesService` uses singleton pattern (`_instance`) because `SharedPreferences.getInstance()` is expensive and should only be called once. The instance is cached in `_prefs`.
- **Type Safety**: Generic methods `get<T>` and `set<T>` provide compile-time type checking. Runtime type checks (`if (T == bool)`) ensure we call the correct SharedPreferences method (`getBool` vs `getInt`).
- **JSON Serialization**: `setObject`/`getObject` handle complex types by serializing to JSON strings. This is necessary because SharedPreferences only supports primitives.
- **Atomic Updates**: The `update` method prevents race conditions when multiple parts of the app modify the same key simultaneously by reading and writing in one operation.
- **Initialization Check**: `_checkInitialized` ensures `init()` was called before any operation, preventing null reference errors.

### **Best Practices and Migration**

```dart
// preferences_migration.dart
// Handling schema changes in preferences over app versions

class PreferencesMigration {
  final PreferencesService _prefs;
  final int currentVersion = 2;  // Increment when changing schema
  
  PreferencesMigration(this._prefs);
  
  /// Run migration if needed
  /// Call this after init() in main()
  Future<void> migrate() async {
    final storedVersion = _prefs.get<int>('prefs_version') ?? 0;
    
    if (storedVersion < currentVersion) {
      print('Migrating preferences from $storedVersion to $currentVersion');
      
      if (storedVersion < 1) await _migrateToV1();
      if (storedVersion < 2) await _migrateToV2();
      
      await _prefs.set<int>('prefs_version', currentVersion);
    }
  }
  
  /// Example: Rename key from 'dark_mode' to 'theme_mode'
  Future<void> _migrateToV1() async {
    final oldValue = _prefs.get<bool>('dark_mode');
    if (oldValue != null) {
      await _prefs.set<String>('theme_mode', oldValue ? 'dark' : 'light');
      await _prefs.remove('dark_mode');
    }
  }
  
  /// Example: Convert single favorite to list
  Future<void> _migrateToV2() async {
    final oldFavorite = _prefs.get<String>('favorite_category');
    if (oldFavorite != null) {
      await _prefs.set<List<String>>('favorite_categories', [oldFavorite]);
      await _prefs.remove('favorite_category');
    }
  }
}
```

**Explanation:**

- **Version Tracking**: Store a schema version number in preferences. When app updates change the preference structure, increment `currentVersion` and add migration logic.
- **Progressive Migration**: The `if (storedVersion < X)` pattern allows skipping migrations for fresh installs while applying all necessary migrations for updates from old versions.
- **Data Transformation**: Migrations can rename keys, change data types (bool to String), or split single values into collections.

---

## **22.2 SQLite with sqflite**

SQLite provides relational database capabilities with SQL support. The `sqflite` package wraps platform-specific implementations (SQLite on iOS/Android, SQL.js on web).

### **Database Setup and CRUD**

```dart
// database_helper.dart
// SQLite database helper with migration support and batch operations

import 'package:sqflite/sqflite.dart';
import 'package:path/path.dart';

/// Database helper for SQLite operations
/// Implements singleton pattern with proper resource management
class DatabaseHelper {
  static final DatabaseHelper instance = DatabaseHelper._init();
  static Database? _database;
  
  // Database configuration
  static const String _databaseName = 'app_database.db';
  static const int _databaseVersion = 2;
  
  DatabaseHelper._init();
  
  /// Get database instance, initializing if necessary
  Future<Database> get database async {
    if (_database != null) return _database!;
    _database = await _initDB();
    return _database!;
  }
  
  /// Initialize database file and open connection
  Future<Database> _initDB() async {
    // Get platform-specific database path
    // iOS: Documents directory
    // Android: databases directory in app storage
    final dbPath = await getDatabasesPath();
    final path = join(dbPath, _databaseName);
    
    return await openDatabase(
      path,
      version: _databaseVersion,
      onCreate: _createDB,      // Called when database is first created
      onUpgrade: _upgradeDB,    // Called when version increases
      onDowngrade: onDatabaseDowngradeDelete,  // Handle downgrades by reset
      // Enable foreign key constraints (disabled by default in SQLite)
      onConfigure: (db) async {
        await db.execute('PRAGMA foreign_keys = ON');
      },
    );
  }
  
  /// Create tables on first install
  Future _createDB(Database db, int version) async {
    const idType = 'INTEGER PRIMARY KEY AUTOINCREMENT';
    const textType = 'TEXT NOT NULL';
    const boolType = 'BOOLEAN NOT NULL CHECK ($textType IN (0, 1))';
    const integerType = 'INTEGER NOT NULL';
    const realType = 'REAL NOT NULL';
    const nullableText = 'TEXT';
    
    // Users table
    await db.execute('''
      CREATE TABLE users (
        id $idType,
        email $textType UNIQUE,
        name $textType,
        age $integerType,
        is_active $boolType DEFAULT 1,
        created_at $textType,
        profile_image_url $nullableText
      )
    ''');
    
    // Posts table with foreign key to users
    await db.execute('''
      CREATE TABLE posts (
        id $idType,
        user_id $integerType,
        title $textType,
        content $textType,
        published_at $textType,
        view_count $integerType DEFAULT 0,
        FOREIGN KEY (user_id) REFERENCES users (id) ON DELETE CASCADE
      )
    ''');
    
    // Indices for performance
    await db.execute('CREATE INDEX idx_posts_user_id ON posts(user_id)');
    await db.execute('CREATE INDEX idx_posts_published ON posts(published_at)');
  }
  
  /// Handle database upgrades
  Future _upgradeDB(Database db, int oldVersion, int newVersion) async {
    if (oldVersion < 2) {
      // Migration to version 2: Add column to existing table
      await db.execute('ALTER TABLE users ADD COLUMN phone_number TEXT');
    }
  }
  
  /// CRUD Operations
  
  /// Create: Insert new record
  /// Returns the auto-generated ID
  Future<int> create(String table, Map<String, dynamic> data) async {
    final db = await instance.database;
    return await db.insert(
      table, 
      data,
      conflictAlgorithm: ConflictAlgorithm.replace,  // Handle unique constraint violations
    );
  }
  
  /// Read: Query with filtering, sorting, and pagination
  Future<List<Map<String, dynamic>>> read({
    required String table,
    List<String>? columns,           // Specific columns to fetch
    String? where,                   // WHERE clause (e.g., 'age > ?')
    List<dynamic>? whereArgs,        // Arguments for WHERE (prevents SQL injection)
    String? orderBy,                 // ORDER BY clause
    int? limit,                      // LIMIT count
    int? offset,                     // OFFSET for pagination
  }) async {
    final db = await instance.database;
    
    return await db.query(
      table,
      columns: columns,
      where: where,
      whereArgs: whereArgs,
      orderBy: orderBy,
      limit: limit,
      offset: offset,
    );
  }
  
  /// Read single record by ID
  Future<Map<String, dynamic>?> readById(String table, int id) async {
    final results = await read(
      table: table,
      where: 'id = ?',
      whereArgs: [id],
      limit: 1,
    );
    return results.isNotEmpty ? results.first : null;
  }
  
  /// Update: Modify existing records
  /// Returns number of rows affected
  Future<int> update({
    required String table,
    required Map<String, dynamic> data,
    required String where,
    required List<dynamic> whereArgs,
  }) async {
    final db = await instance.database;
    return await db.update(
      table,
      data,
      where: where,
      whereArgs: whereArgs,
    );
  }
  
  /// Delete: Remove records
  /// Returns number of rows deleted
  Future<int> delete({
    required String table,
    required String where,
    required List<dynamic> whereArgs,
  }) async {
    final db = await instance.database;
    return await db.delete(
      table,
      where: where,
      whereArgs: whereArgs,
    );
  }
  
  /// Batch operations for performance
  /// Use for bulk inserts/updates
  Future<void> batchInsert(String table, List<Map<String, dynamic>> records) async {
    final db = await instance.database;
    
    await db.transaction((txn) async {
      final batch = txn.batch();
      
      for (final record in records) {
        batch.insert(table, record, conflictAlgorithm: ConflictAlgorithm.replace);
      }
      
      await batch.commit(noResult: true);  // noResult: true for better performance
    });
  }
  
  /// Raw SQL execution (use sparingly, prefer type-safe methods)
  Future<List<Map<String, dynamic>>> rawQuery(String sql, [List<dynamic>? arguments]) async {
    final db = await instance.database;
    return await db.rawQuery(sql, arguments);
  }
  
  /// Close database connection
  /// Call this when app is terminating or database no longer needed
  Future close() async {
    final db = await instance.database;
    await db.close();
    _database = null;
  }
}
```

**Explanation:**

- **`getDatabasesPath()`**: Returns platform-appropriate directory. On iOS, this is the `Documents` directory (backed up to iCloud). On Android, it's the app-private databases directory.
- **`ConflictAlgorithm.replace`**: If inserting a record violates a unique constraint (e.g., duplicate email), this replaces the existing record instead of throwing an error.
- **Foreign Keys**: `PRAGMA foreign_keys = ON` enables SQLite foreign key constraints, which are disabled by default for backward compatibility. `ON DELETE CASCADE` automatically deletes posts when their user is deleted.
- **Indices**: `CREATE INDEX` creates database indices for faster queries on frequently filtered columns (`user_id`, `published_at`), at the cost of slightly slower writes and more storage.
- **Parameterized Queries**: `whereArgs` automatically escapes strings and prevents SQL injection. Never concatenate user input directly into SQL strings.
- **Batch Operations**: `Batch` groups multiple operations into a single transaction, dramatically improving performance for bulk inserts (1000x faster than individual inserts).

### **Model Classes and Repository Pattern**

```dart
// models/user.dart
// Immutable data model with JSON serialization

class User {
  final int? id;  // Nullable for new records not yet inserted
  final String email;
  final String name;
  final int age;
  final bool isActive;
  final DateTime createdAt;
  final String? profileImageUrl;
  final String? phoneNumber;  // Added in v2
  
  User({
    this.id,
    required this.email,
    required this.name,
    required this.age,
    this.isActive = true,
    required this.createdAt,
    this.profileImageUrl,
    this.phoneNumber,
  });
  
  /// Convert from database map (snake_case keys)
  factory User.fromMap(Map<String, dynamic> map) {
    return User(
      id: map['id'] as int?,
      email: map['email'] as String,
      name: map['name'] as String,
      age: map['age'] as int,
      isActive: map['is_active'] == 1,  // SQLite stores bool as 0/1
      createdAt: DateTime.parse(map['created_at'] as String),
      profileImageUrl: map['profile_image_url'] as String?,
      phoneNumber: map['phone_number'] as String?,
    );
  }
  
  /// Convert to database map
  Map<String, dynamic> toMap() {
    return {
      if (id != null) 'id': id,  // Only include if not null (auto-increment)
      'email': email,
      'name': name,
      'age': age,
      'is_active': isActive ? 1 : 0,
      'created_at': createdAt.toIso8601String(),
      'profile_image_url': profileImageUrl,
      'phone_number': phoneNumber,
    };
  }
  
  /// Copy with method for immutable updates
  User copyWith({
    int? id,
    String? email,
    String? name,
    int? age,
    bool? isActive,
    DateTime? createdAt,
    String? profileImageUrl,
    String? phoneNumber,
  }) {
    return User(
      id: id ?? this.id,
      email: email ?? this.email,
      name: name ?? this.name,
      age: age ?? this.age,
      isActive: isActive ?? this.isActive,
      createdAt: createdAt ?? this.createdAt,
      profileImageUrl: profileImageUrl ?? this.profileImageUrl,
      phoneNumber: phoneNumber ?? this.phoneNumber,
    );
  }
}

// repositories/user_repository.dart
// Repository pattern for data access abstraction

class UserRepository {
  final DatabaseHelper _dbHelper;
  
  UserRepository(this._dbHelper);
  
  Future<User> createUser(User user) async {
    final id = await _dbHelper.create('users', user.toMap());
    return user.copyWith(id: id);
  }
  
  Future<User?> getUser(int id) async {
    final map = await _dbHelper.readById('users', id);
    return map != null ? User.fromMap(map) : null;
  }
  
  Future<User?> getUserByEmail(String email) async {
    final results = await _dbHelper.read(
      table: 'users',
      where: 'email = ?',
      whereArgs: [email],
      limit: 1,
    );
    return results.isNotEmpty ? User.fromMap(results.first) : null;
  }
  
  Future<List<User>> getAllUsers({bool activeOnly = false}) async {
    final results = await _dbHelper.read(
      table: 'users',
      where: activeOnly ? 'is_active = ?' : null,
      whereArgs: activeOnly ? [1] : null,
      orderBy: 'created_at DESC',
    );
    return results.map((m) => User.fromMap(m)).toList();
  }
  
  Future<int> updateUser(User user) async {
    if (user.id == null) throw ArgumentError('Cannot update user without ID');
    
    return await _dbHelper.update(
      table: 'users',
      data: user.toMap(),
      where: 'id = ?',
      whereArgs: [user.id],
    );
  }
  
  Future<int> deleteUser(int id) async {
    return await _dbHelper.delete(
      table: 'users',
      where: 'id = ?',
      whereArgs: [id],
    );
  }
  
  /// Complex query: Search users with pagination
  Future<List<User>> searchUsers({
    String? query,
    int? minAge,
    int? maxAge,
    required int page,
    required int pageSize,
  }) async {
    final conditions = <String>[];
    final args = <dynamic>[];
    
    if (query != null && query.isNotEmpty) {
      conditions.add('(name LIKE ? OR email LIKE ?)');
      args.add('%$query%');
      args.add('%$query%');
    }
    
    if (minAge != null) {
      conditions.add('age >= ?');
      args.add(minAge);
    }
    
    if (maxAge != null) {
      conditions.add('age <= ?');
      args.add(maxAge);
    }
    
    final whereClause = conditions.isNotEmpty ? conditions.join(' AND ') : null;
    
    final results = await _dbHelper.read(
      table: 'users',
      where: whereClause,
      whereArgs: args.isNotEmpty ? args : null,
      orderBy: 'name ASC',
      limit: pageSize,
      offset: (page - 1) * pageSize,
    );
    
    return results.map((m) => User.fromMap(m)).toList();
  }
}
```

**Explanation:**

- **Immutable Models**: `User` is immutable (all fields final). Updates create new instances via `copyWith`, preventing accidental state mutations.
- **Bool Conversion**: SQLite has no native boolean type; `0` = false, `1` = true. The `fromMap`/`toMap` methods handle this conversion.
- **Repository Pattern**: `UserRepository` abstracts database operations, allowing the UI to work with domain objects (`User`) rather than database maps. This makes testing easier (you can mock the repository) and allows swapping SQLite for a different backend later.
- **Dynamic Query Building**: The `searchUsers` method programmatically builds SQL WHERE clauses based on optional parameters, demonstrating how to handle complex filtering without string concatenation vulnerabilities.

---

## **22.3 Type-Safe ORM (Drift)**

Drift (formerly Moor) is a reactive persistence library for Dart and Flutter with built-in support for streaming queries, type safety, and compile-time SQL verification.

### **Drift Setup and Usage**

```dart
// database/tables.dart
// Define database tables using Drift DSL

import 'package:drift/drift.dart';

/// Users table definition
/// Drift generates a User class and UsersCompanion for inserts/updates
class Users extends Table {
  IntColumn get id => integer().autoIncrement()();
  TextColumn get email => text().unique()();
  TextColumn get name => text().withLength(min: 1, max: 100)();
  IntColumn get age => integer().check(age.isBiggerThan(const Constant(0)))();
  BoolColumn get isActive => boolean().withDefault(const Constant(true))();
  DateTimeColumn get createdAt => dateTime().withDefault(currentDateAndTime)();
  TextColumn get profileImageUrl => text().nullable()();
}

/// Posts table with foreign key relationship
class Posts extends Table {
  IntColumn get id => integer().autoIncrement()();
  
  // Foreign key to users table
  IntColumn get userId => integer().references(Users, #id, onDelete: KeyAction.cascade)();
  
  TextColumn get title => text().withLength(min: 1, max: 200)();
  TextColumn get content => text()();
  DateTimeColumn get publishedAt => dateTime().nullable()();
  IntColumn get viewCount => integer().withDefault(const Constant(0))();
}

// database/database.dart
// Database configuration and DAO setup

import 'dart:io';
import 'package:drift/drift.dart';
import 'package:drift/native.dart';
import 'package:path_provider/path_provider.dart';
import 'package:path/path.dart' as p;
import 'tables.dart';

part 'database.g.dart';  // Generated file

/// Drift database class
/// Generate code with: flutter pub run build_runner build
@DriftDatabase(tables: [Users, Posts], daos: [UserDao, PostDao])
class AppDatabase extends _$AppDatabase {
  AppDatabase() : super(_openConnection());
  
  @override
  int get schemaVersion => 2;
  
  @override
  MigrationStrategy get migration => MigrationStrategy(
    onCreate: (Migrator m) async {
      await m.createAll();
    },
    onUpgrade: (Migrator m, int from, int to) async {
      if (from < 2) {
        // Migration example: Add column
        await m.addColumn(users, users.phoneNumber);
      }
    },
  );
  
  static QueryExecutor _openConnection() {
    return LazyDatabase(() async {
      final dbFolder = await getApplicationDocumentsDirectory();
      final file = File(p.join(dbFolder.path, 'db.sqlite'));
      return NativeDatabase(file);
    });
  }
  
  /// Close database connection
  Future<void> closeConnection() => close();
}

/// Data Access Object for Users
/// Encapsulates user-specific queries
@DriftAccessor(tables: [Users])
class UserDao extends DatabaseAccessor<AppDatabase> with _$UserDaoMixin {
  UserDao(AppDatabase db) : super(db);
  
  /// Stream of all active users
  /// Automatically emits new values when data changes
  Stream<List<User>> watchAllActiveUsers() {
    return (select(users)..where((u) => u.isActive.equals(true))).watch();
  }
  
  /// Get single user by ID
  Future<User?> getUserById(int id) {
    return (select(users)..where((u) => u.id.equals(id))).getSingleOrNull();
  }
  
  /// Insert user
  Future<int> insertUser(UsersCompanion user) {
    return into(users).insert(user);
  }
  
  /// Update user
  Future<bool> updateUser(User user) {
    return update(users).replace(user);
  }
  
  /// Delete user
  Future<int> deleteUser(int id) {
    return (delete(users)..where((u) => u.id.equals(id))).go();
  }
  
  /// Complex query with joins
  Future<List<UserWithPostCount>> getUsersWithPostCounts() async {
    final query = select(users).join([
      leftOuterJoin(
        posts,
        posts.userId.equalsExp(users.id),
      ),
    ]);
    
    final result = await query.map((row) {
      final user = row.readTable(users);
      final postCount = row.read(posts.id.count());
      
      return UserWithPostCount(user: user, postCount: postCount ?? 0);
    }).get();
    
    return result;
  }
}

/// Data Access Object for Posts
@DriftAccessor(tables: [Posts])
class PostDao extends DatabaseAccessor<AppDatabase> with _$PostDaoMixin {
  PostDao(AppDatabase db) : super(db);
  
  /// Get posts by user with pagination
  Future<List<Post>> getPostsByUser(int userId, {int limit = 20, int offset = 0}) {
    return (select(posts)
      ..where((p) => p.userId.equals(userId))
      ..orderBy([(p) => OrderingTerm.desc(p.publishedAt)])
      ..limit(limit, offset: offset))
      .get();
  }
  
  /// Increment view count atomically
  Future<int> incrementViewCount(int postId) {
    return update(posts).write(
      PostsCompanion.custom(
        viewCount: posts.viewCount + const Constant(1),
      ),
    );
  }
}

/// Custom result class for joined queries
class UserWithPostCount {
  final User user;
  final int postCount;
  
  UserWithPostCount({required this.user, required this.postCount});
}
```

**Explanation:**

- **Type-Safe DSL**: `IntColumn`, `TextColumn`, etc., provide compile-time type safety. Constraints like `unique()`, `check()`, and `references()` are validated at compile time.
- **Code Generation**: The `@DriftDatabase` annotation triggers code generation (`database.g.dart`) that creates type-safe query methods, table classes, and data classes.
- **Reactive Streams**: `watch()` returns a `Stream` that automatically emits new data whenever the underlying table changes. This integrates seamlessly with Flutter's `StreamBuilder` for real-time UI updates.
- **Companions**: `UsersCompanion` is generated for inserts/updates, allowing partial objects (e.g., updating only the name without fetching the full row).
- **Custom Expressions**: `posts.viewCount + const Constant(1)` generates SQL `view_count + 1` for atomic increments without race conditions.
- **Migrations**: Drift's migration system tracks schema versions and provides type-safe ways to add columns, create tables, or transform data during upgrades.

---

## **22.4 Hive NoSQL Database**

Hive is a lightweight, high-performance key-value database written in pure Dart. It stores data in binary format, making it significantly faster than SQLite for simple read/write operations and ideal for caching.

### **Hive Implementation**

```dart
// hive_service.dart
// Hive implementation with type adapters and encryption

import 'package:hive/hive.dart';
import 'package:hive_flutter/hive_flutter.dart';
import 'package:flutter/foundation.dart';
import 'dart:convert';
import 'dart:typed_data';

/// Hive service for high-performance local storage
/// Best for: Caching, user settings, simple object storage
/// Not suitable for: Complex queries, relational data, large datasets (>1GB)
class HiveService {
  static final HiveService _instance = HiveService._internal();
  factory HiveService() => _instance;
  HiveService._internal();
  
  bool _initialized = false;
  
  /// Initialize Hive
  /// Call before using any boxes
  Future<void> init() async {
    if (_initialized) return;
    
    // Initialize Hive Flutter (sets up proper path)
    await Hive.initFlutter();
    
    // Register type adapters for custom objects
    // Generated using: flutter pub run build_runner build
    Hive.registerAdapter(UserModelAdapter());
    Hive.registerAdapter(SettingsAdapter());
    
    _initialized = true;
  }
  
  /// Open a box (collection) with optional encryption
  /// [name]: Box name (e.g., 'users', 'cache')
  /// [encryptionKey]: 32-byte key for AES-256 encryption
  Future<Box<T>> openBox<T>(String name, {Uint8List? encryptionKey}) async {
    _checkInitialized();
    
    return await Hive.openBox<T>(
      name,
      encryptionCipher: encryptionKey != null 
          ? HiveAesCipher(encryptionKey) 
          : null,
      compactionStrategy: (entries, deletedEntries) {
        // Compact when 20% of entries are deleted
        // Compaction rewrites the file to remove dead space
        return deletedEntries > entries * 0.2;
      },
    );
  }
  
  /// Open lazy box (loads keys only, values on demand)
  /// Better for large datasets where you don't need all data in memory
  Future<LazyBox<T>> openLazyBox<T>(String name) async {
    _checkInitialized();
    return await Hive.openLazyBox<T>(name);
  }
  
  /// Generic CRUD operations
  
  Future<void> put<T>(String boxName, String key, T value) async {
    final box = await openBox<T>(boxName);
    await box.put(key, value);
  }
  
  Future<T?> get<T>(String boxName, String key) async {
    final box = await openBox<T>(boxName);
    return box.get(key);
  }
  
  Future<void> delete(String boxName, String key) async {
    final box = await openBox(boxName);
    await box.delete(key);
  }
  
  Future<List<T>> getAll<T>(String boxName) async {
    final box = await openBox<T>(boxName);
    return box.values.toList();
  }
  
  /// Watch specific key for changes
  Stream<T?> watchKey<T>(String boxName, String key) async* {
    final box = await openBox<T>(boxName);
    yield box.get(key);
    yield* box.watch(key: key).map((event) => event.value as T?);
  }
  
  /// Watch entire box for changes
  Stream<BoxEvent> watchBox(String boxName) async* {
    final box = await openBox(boxName);
    yield* box.watch();
  }
  
  /// Clear box contents
  Future<void> clearBox(String boxName) async {
    final box = await openBox(boxName);
    await box.clear();
  }
  
  /// Close specific box
  Future<void> closeBox(String boxName) async {
    if (Hive.isBoxOpen(boxName)) {
      final box = Hive.box(boxName);
      await box.close();
    }
  }
  
  /// Close all boxes and Hive
  Future<void> closeAll() async {
    await Hive.close();
  }
  
  void _checkInitialized() {
    if (!_initialized) throw StateError('HiveService not initialized');
  }
}

/// Example model with Hive type adapter
/// Run: flutter pub run build_runner build to generate adapter
@HiveType(typeId: 0)  // Unique ID for this class
class UserModel extends HiveObject {
  @HiveField(0)
  final String id;
  
  @HiveField(1)
  final String email;
  
  @HiveField(2)
  final String name;
  
  @HiveField(3)
  final DateTime lastSync;
  
  @HiveField(4, defaultValue: false)
  final bool isSynced;
  
  UserModel({
    required this.id,
    required this.email,
    required this.name,
    required this.lastSync,
    this.isSynced = false,
  });
  
  Map<String, dynamic> toJson() => {
    'id': id,
    'email': email,
    'name': name,
    'lastSync': lastSync.toIso8601String(),
    'isSynced': isSynced,
  };
  
  factory UserModel.fromJson(Map<String, dynamic> json) => UserModel(
    id: json['id'],
    email: json['email'],
    name: json['name'],
    lastSync: DateTime.parse(json['lastSync']),
    isSynced: json['isSynced'] ?? false,
  );
}

/// Generated adapter (normally in user_model.g.dart)
class UserModelAdapter extends TypeAdapter<UserModel> {
  @override
  final int typeId = 0;

  @override
  UserModel read(BinaryReader reader) {
    final numOfFields = reader.readByte();
    final fields = <int, dynamic>{
      for (int i = 0; i < numOfFields; i++) reader.readByte(): reader.read(),
    };
    return UserModel(
      id: fields[0] as String,
      email: fields[1] as String,
      name: fields[2] as String,
      lastSync: fields[3] as DateTime,
      isSynced: fields[4] == null ? false : fields[4] as bool,
    );
  }

  @override
  void write(BinaryWriter writer, UserModel obj) {
    writer
      ..writeByte(5)
      ..writeByte(0)
      ..write(obj.id)
      ..writeByte(1)
      ..write(obj.email)
      ..writeByte(2)
      ..write(obj.name)
      ..writeByte(3)
      ..write(obj.lastSync)
      ..writeByte(4)
      ..write(obj.isSynced);
  }
}

/// Secure key generation for Hive encryption
class HiveEncryption {
  /// Generate secure 32-byte (256-bit) key for AES encryption
  /// Store this securely (Keychain/Keystore), not in SharedPreferences
  static Uint8List generateSecureKey() {
    return Hive.generateSecureKey();
  }
  
  /// Convert key to base64 for storage
  static String keyToBase64(Uint8List key) {
    return base64Encode(key);
  }
  
  /// Convert base64 back to key
  static Uint8List keyFromBase64(String base64Key) {
    return base64Decode(base64Key);
  }
}
```

**Explanation:**

- **Binary Storage**: Hive stores data in binary format using custom serialization, making it 5-10x faster than SQLite for simple key-value operations and JSON parsing.
- **Type Adapters**: The `@HiveType` and `@HiveField` annotations generate efficient binary serialization code. `typeId` must be unique across all adapters in the app.
- **Lazy Boxes**: `LazyBox` keeps only keys in memory, loading values from disk on demand. Essential for large datasets (thousands of items) to prevent memory issues.
- **Encryption**: `HiveAesCipher` provides AES-256 encryption. The encryption key must be 32 bytes and stored securely (not in Hive itself). Use `flutter_secure_storage` to store the encryption key.
- **Compaction**: Hive is append-only (writes new data to end of file). `compactionStrategy` triggers file rewriting to remove deleted entries when waste exceeds 20%.
- **Box vs. Table**: Unlike SQL tables, Hive boxes are schemaless. You can store different types in one box, but it's best practice to separate by type for type safety.

---

## **22.5 ObjectBox & Isar**

ObjectBox and Isar are high-performance NoSQL databases optimized for Flutter. ObjectBox focuses on object graphs and relations, while Isar (built by the same author as Hive) offers the fastest queries in Flutter with full-text search and composite indices.

### **Isar Database Implementation**

```dart
// isar_collections.dart
// Define Isar collections (similar to Hive but with queries)

import 'package:isar/isar.dart';

part 'isar_collections.g.dart';  // Generated

/// User collection with indices for fast queries
@collection
class IsarUser {
  Id id = Isar.autoIncrement;  // Auto-increment primary key
  
  @Index(unique: true)  // Unique constraint
  late String email;
  
  @Index()  // Index for fast sorting/filtering
  late String name;
  
  late int age;
  
  @Index(composite: [CompositeIndex('age')])  // Composite index
  late DateTime createdAt;
  
  bool isActive = true;
  
  final posts = IsarLinks<IsarPost>();  // One-to-many relation
  
  @ignore  // Not stored in database
  String? temporaryToken;
}

@collection
class IsarPost {
  Id id = Isar.autoIncrement;
  
  late String title;
  
  @Index(type: IndexType.value)  // Index for full-text search
  late String content;
  
  late DateTime publishedAt;
  int viewCount = 0;
  
  final user = IsarLink<IsarUser>();  // Backlink to user
}

// isar_service.dart
// Isar database service with advanced queries

import 'package:isar/isar.dart';
import 'package:path_provider/path_provider.dart';
import 'isar_collections.dart';

class IsarService {
  static Isar? _isar;
  
  /// Initialize Isar
  static Future<void> initialize() async {
    if (_isar != null) return;
    
    final dir = await getApplicationDocumentsDirectory();
    
    _isar = await Isar.open(
      [IsarUserSchema, IsarPostSchema],
      directory: dir.path,
      inspector: true,  // Enable DevTools inspector
    );
  }
  
  static Isar get instance {
    if (_isar == null) throw StateError('Isar not initialized');
    return _isar!;
  }
  
  /// CRUD Operations
  
  Future<int> saveUser(IsarUser user) async {
    return await instance.writeTxn(() async {
      return await instance.isarUsers.put(user);
    });
  }
  
  Future<IsarUser?> getUser(int id) async {
    return await instance.isarUsers.get(id);
  }
  
  Future<List<IsarUser>> getAllUsers() async {
    return await instance.isarUsers.where().findAll();
  }
  
  /// Advanced Queries
  
  /// Get users with filtering, sorting, pagination
  Future<List<IsarUser>> queryUsers({
    String? nameContains,
    int? minAge,
    int? maxAge,
    bool? isActive,
    String? sortBy,
    bool ascending = true,
    int offset = 0,
    int limit = 50,
  }) async {
    var query = instance.isarUsers.where();
    
    // Filters
    if (nameContains != null) {
      query = query.filter().nameContains(nameContains) as QueryBuilder<IsarUser, IsarUser, QWhere>;
    }
    
    if (minAge != null) {
      query = query.filter().ageGreaterThan(minAge - 1) as QueryBuilder<IsarUser, IsarUser, QWhere>;
    }
    
    if (maxAge != null) {
      query = query.filter().ageLessThan(maxAge + 1) as QueryBuilder<IsarUser, IsarUser, QWhere>;
    }
    
    if (isActive != null) {
      query = query.filter().isActiveEqualTo(isActive) as QueryBuilder<IsarUser, IsarUser, QWhere>;
    }
    
    // Sorting
    QueryBuilder<IsarUser, IsarUser, QAfterFilterCondition> sortedQuery;
    if (sortBy == 'name') {
      sortedQuery = ascending 
          ? (query as QueryBuilder<IsarUser, IsarUser, QAfterFilterCondition>).sortByName()
          : (query as QueryBuilder<IsarUser, IsarUser, QAfterFilterCondition>).sortByNameDesc();
    } else if (sortBy == 'age') {
      sortedQuery = ascending
          ? (query as QueryBuilder<IsarUser, IsarUser, QAfterFilterCondition>).sortByAge()
          : (query as QueryBuilder<IsarUser, IsarUser, QAfterFilterCondition>).sortByAgeDesc();
    } else {
      sortedQuery = query as QueryBuilder<IsarUser, IsarUser, QAfterFilterCondition>;
    }
    
    // Pagination
    return await sortedQuery.offset(offset).limit(limit).findAll();
  }
  
  /// Full-text search in posts
  Future<List<IsarPost>> searchPosts(String query) async {
    return await instance.isarPosts
        .filter()
        .contentContains(query, caseSensitive: false)
        .sortByPublishedAtDesc()
        .findAll();
  }
  
  /// Atomic increment
  Future<void> incrementViewCount(int postId) async {
    await instance.writeTxn(() async {
      final post = await instance.isarPosts.get(postId);
      if (post != null) {
        post.viewCount++;
        await instance.isarPosts.put(post);
      }
    });
  }
  
  /// Relations: Get user with all posts
  Future<IsarUser?> getUserWithPosts(int userId) async {
    final user = await instance.isarUsers.get(userId);
    if (user != null) {
      await user.posts.load();  // Lazy load relation
    }
    return user;
  }
  
  /// Watch query for real-time updates
  Stream<List<IsarUser>> watchActiveUsers() {
    return instance.isarUsers
        .filter()
        .isActiveEqualTo(true)
        .watch(fireImmediately: true);
  }
  
  /// Aggregation
  Future<int> getTotalViewCount() async {
    return await instance.isarPosts.where().viewCountProperty().sum();
  }
  
  /// Batch operations
  Future<void> saveUsers(List<IsarUser> users) async {
    await instance.writeTxn(() async {
      await instance.isarUsers.putAll(users);
    });
  }
  
  /// Delete with cascade
  Future<void> deleteUser(int userId) async {
    await instance.writeTxn(() async {
      // Delete related posts first (or configure cascade in schema)
      final user = await instance.isarUsers.get(userId);
      if (user != null) {
        await user.posts.load();
        await instance.isarPosts.deleteAll(user.posts.map((p) => p.id).toList());
        await instance.isarUsers.delete(userId);
      }
    });
  }
  
  /// Close database
  Future<void> close() async {
    await _isar?.close();
    _isar = null;
  }
}
```

**Explanation:**

- **Indices**: `@Index()` creates B-tree indices for O(log n) lookups. `@Index(unique: true)` enforces uniqueness. Composite indices optimize multi-column queries.
- **Relations**: `IsarLinks` and `IsarLink` define relationships between objects. Unlike SQL foreign keys, these are object references loaded lazily with `.load()`.
- **Type-Safe Queries**: Isar generates query methods like `filter().nameContains()` and `sortByAgeDesc()` with compile-time verification, eliminating SQL injection risks.
- **Watchers**: `.watch()` returns streams that emit on database changes, enabling reactive UI patterns similar to Drift but with better performance for large datasets.
- **Transactions**: `writeTxn()` ensures ACID compliance. All write operations inside the transaction succeed or fail together.
- **Full-Text Search**: `contentContains` performs case-insensitive substring searches efficiently using indices.

---

## **22.6 File System Operations**

For large binary data (images, videos, documents), use the file system instead of databases.

### **File Storage Service**

```dart
// file_storage_service.dart
// Secure file operations with path management

import 'dart:io';
import 'dart:typed_data';
import 'package:path_provider/path_provider.dart';
import 'package:path/path.dart' as p;
import 'package:crypto/crypto.dart';
import 'package:mime/mime.dart';

/// Service for file system operations
/// Handles images, documents, cache management
class FileStorageService {
  /// Get app documents directory (backed up to iCloud/iTunes)
  Future<Directory> get documentsDir async {
    return await getApplicationDocumentsDirectory();
  }
  
  /// Get temporary directory (system may clear this)
  Future<Directory> get tempDir async {
    return await getTemporaryDirectory();
  }
  
  /// Get cache directory (for images, network responses)
  Future<Directory> get cacheDir async {
    return await getApplicationCacheDirectory();
  }
  
  /// Save file with automatic subdirectory creation
  /// [relativePath]: Path relative to documents dir (e.g., 'images/profile.png')
  /// [data]: File bytes
  /// [isTemporary]: If true, saves to temp dir instead of documents
  Future<File> saveFile(
    String relativePath,
    Uint8List data, {
    bool isTemporary = false,
  }) async {
    final baseDir = isTemporary ? await tempDir : await documentsDir;
    final filePath = p.join(baseDir.path, relativePath);
    
    // Create subdirectories if needed
    final dir = Directory(p.dirname(filePath));
    if (!await dir.exists()) {
      await dir.create(recursive: true);
    }
    
    final file = File(filePath);
    await file.writeAsBytes(data, flush: true);  // flush ensures written to disk
    return file;
  }
  
  /// Save file with content hash as filename (deduplication)
  /// Returns existing path if file with same hash already exists
  Future<String> saveFileWithHash(
    Uint8List data,
    String extension, {
    String subdirectory = 'cache',
  }) async {
    final hash = sha256.convert(data).toString();
    final filename = '$hash.$extension';
    final relativePath = p.join(subdirectory, filename);
    
    final existingPath = await getFilePath(relativePath);
    if (existingPath != null) {
      return existingPath;  // File already exists
    }
    
    final file = await saveFile(relativePath, data);
    return file.path;
  }
  
  /// Read file as bytes
  Future<Uint8List?> readFile(String relativePath) async {
    final path = await getFilePath(relativePath);
    if (path == null) return null;
    
    try {
      return await File(path).readAsBytes();
    } catch (e) {
      return null;
    }
  }
  
  /// Get absolute path if file exists
  Future<String?> getFilePath(String relativePath) async {
    final baseDir = await documentsDir;
    final fullPath = p.join(baseDir.path, relativePath);
    
    final file = File(fullPath);
    return await file.exists() ? fullPath : null;
  }
  
  /// Delete file
  Future<bool> deleteFile(String relativePath) async {
    final path = await getFilePath(relativePath);
    if (path == null) return false;
    
    try {
      await File(path).delete();
      return true;
    } catch (e) {
      return false;
    }
  }
  
  /// Clear cache directory (call periodically or on low memory)
  Future<void> clearCache() async {
    final dir = await cacheDir;
    await _deleteContents(dir);
  }
  
  /// Get directory size for storage monitoring
  Future<int> getDirectorySize(String relativePath) async {
    final baseDir = await documentsDir;
    final dir = Directory(p.join(baseDir.path, relativePath));
    
    if (!await dir.exists()) return 0;
    
    int totalSize = 0;
    await for (final entity in dir.list(recursive: true)) {
      if (entity is File) {
        totalSize += await entity.length();
      }
    }
    return totalSize;
  }
  
  /// List files in directory with metadata
  Future<List<FileInfo>> listFiles(String relativePath) async {
    final baseDir = await documentsDir;
    final dir = Directory(p.join(baseDir.path, relativePath));
    
    if (!await dir.exists()) return [];
    
    final files = <FileInfo>[];
    await for (final entity in dir.list()) {
      if (entity is File) {
        final stat = await entity.stat();
        files.add(FileInfo(
          path: entity.path,
          name: p.basename(entity.path),
          size: stat.size,
          modified: stat.modified,
          mimeType: lookupMimeType(entity.path),
        ));
      }
    }
    return files;
  }
  
  /// Helper: Delete directory contents
  Future<void> _deleteContents(Directory dir) async {
    await for (final entity in dir.list()) {
      await entity.delete(recursive: true);
    }
  }
}

class FileInfo {
  final String path;
  final String name;
  final int size;
  final DateTime modified;
  final String? mimeType;
  
  FileInfo({
    required this.path,
    required this.name,
    required this.size,
    required this.modified,
    this.mimeType,
  });
  
  String get sizeFormatted {
    if (size < 1024) return '$size B';
    if (size < 1024 * 1024) return '${(size / 1024).toStringAsFixed(1)} KB';
    return '${(size / (1024 * 1024)).toStringAsFixed(1)} MB';
  }
}
```

**Explanation:**

- **Path Providers**: `getApplicationDocumentsDirectory()` returns persistent storage (backed up by iCloud/iTunes). `getTemporaryDirectory()` returns cache that OS may clear. `getApplicationCacheDirectory()` is for user-generated cache (images).
- **SHA-256 Deduplication**: Storing files by content hash prevents duplicate storage of identical images/files. Checking existence before writing saves disk space.
- **MIME Type Detection**: The `mime` package detects file types from magic numbers/file extensions for proper HTTP content-type headers when sharing files.
- **Recursive Creation**: `dir.create(recursive: true)` creates parent directories automatically, preventing "directory not found" errors.
- **Flush to Disk**: `writeAsBytes(data, flush: true)` ensures data is physically written to storage before returning, preventing data loss if app crashes immediately after write.

---

## **Chapter Summary**

In this chapter, we covered local data persistence strategies for Flutter:

### **Key Takeaways:**

1. **SharedPreferences**: Use only for simple primitives (booleans, small strings) and user settings. Never store sensitive data or large objects. Implement migration strategies for schema changes.

2. **SQLite (sqflite)**: Best for relational data requiring complex queries, joins, and ACID compliance. Use parameterized queries (`whereArgs`) to prevent SQL injection. Implement Repository pattern to abstract database details from UI.

3. **Drift (Moor)**: Type-safe ORM that generates SQL at compile time. Provides reactive streams (`watch()`) for automatic UI updates. Use for applications requiring complex database schemas with type safety.

4. **Hive**: High-performance NoSQL for simple key-value storage and caching. Binary format is 5-10x faster than SQLite for simple reads. Use LazyBoxes for large datasets. Enable AES-256 encryption for sensitive data.

5. **Isar**: Successor to Hive with query capabilities. Fastest Flutter database for complex queries with full-text search and composite indices. Best for object graphs with relations.

6. **File System**: Store large binary data (images, videos) in files, not databases. Use content hashing for deduplication. Clear cache directories periodically to manage storage.

### **Selection Guide:**

- **Settings/Flags**: SharedPreferences
- **Relational data/Reports**: SQLite/Drift
- **Cache/Simple objects**: Hive
- **Complex queries/Search**: Isar
- **Images/Videos**: File System + path_provider

### **Security Considerations:**

- Never store passwords or tokens in SharedPreferences (use flutter_secure_storage)
- Encrypt Hive boxes containing PII (Personally Identifiable Information)
- Validate file paths to prevent directory traversal attacks
- Clear cache on logout to prevent data leakage

### **Next Steps:**

The next chapter (Chapter 23) will cover **Platform Integration**, including:
- Platform channels for native code communication
- MethodChannel and EventChannel implementation
- PlatformView for embedding native UI components
- Platform-specific plugin development
- FFI (Foreign Function Interface) for C/C++ interop

---

**End of Chapter 22**